In [1]:
load('Imports/Utility.sage', 'Imports/Point.sage', 'Imports/Line.sage', 'Imports/Tritangent_plane.sage', 'Imports/Group.sage', 'Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [2]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(QQ)

In [3]:
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)

In [4]:
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f)^2 * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)

In [5]:
line = Line([y, z])

In [6]:
eqn_SE1 = P(cubic_new.subs({b:-(c*c+e*f)/c }) )

In [7]:
sing_cubics_SE1 = P(sing_cubics.subs({b:-(c*c+e*f)/c }))

In [8]:
SE1 = Cubic(eqn_SE1, line, sing_cubics_SE1.factor())

In [9]:
%time adm_SE1 = SE1.find_admissible_projectivities()

CPU times: user 1.25 s, sys: 157 ms, total: 1.41 s
Wall time: 10.3 s


In [10]:
len(adm_SE1)

576

In [11]:
%time simm_SE1 = SE1.find_simmetries(adm_SE1)

CPU times: user 1.07 s, sys: 69.3 ms, total: 1.14 s
Wall time: 4.67 s


In [12]:
len(simm_SE1)

2

In [13]:
G_SE1 = Group(simm_SE1)

In [14]:
G_SE1.is_group()

True

In [15]:
G_SE1.get_order()

2

In [16]:
G_SE1.is_abelian()

True

In [17]:
G_SE1.get_divisors_of_order()

[1, 2]

In [18]:
G_SE1.get_order_of_elements()

[1, 2]

In [19]:
G_SE1.apply_to(SE1.eckardt_points[0])

[(0, 0, 0, 1), (0, 0, 0, 1)]

In [ ]:
def find_conditions_for_subfamilies(cubic, sing_cubics, projectivities, simmetries):
    mon = ((x+y+z+t)^3).monomials()
    conditions = []
    sing_cubics_factored = sing_cubics.factor()
    for M in [proj for proj in projectivities if proj not in simmetries]:
        sost = change_coord(M)
        new_cubic = remove_sing_factors(cubic.eqn.subs(sost), sing_cubics_factored)    
        minor = matrix([[new_cubic.coefficient(mn) for mn in mon], [cubic.eqn.coefficient(mn) for mn in mon]]).minors(2)
        minor = list(set([remove_sing_factors(el, sing_cubics_factored) for el in minor if el !=0]))
        ideale = cubic.P.ideal(minor)
        new_ideals = get_valid_ideals(ideale, cubic, sing_cubics_factored)
        for ideal in new_ideals:            
            prim_deco2 = (ideale + ideal).radical().primary_decomposition()
            for ideal2 in prim_deco2:
                if is_ideal_valid(ideal2, cubic, sing_cubics_factored):
                    conditions.append(ideal2.gens())                        
    return list(set(conditions))


def get_valid_ideals(ideale, cubic, sing_cubics_factored):
    valid_ideals = []
    vrs = [c,d,e,f]
    for vr in vrs:
        valid_ideals_var = []
        del_ideale = ideale.elimination_ideal([vr])
        prim_deco_del = del_ideale.radical().primary_decomposition()
        for ideal in prim_deco_del:
            if is_ideal_valid(ideal, cubic, sing_cubics_factored):
                valid_ideals_var.append(ideal)
        if valid_ideals_var == []:
            return []
        else:
            valid_ideals.append(valid_ideals_var)
    #return [id1+id2+id3 for id1 in valid_ideals[0] for id2 in valid_ideals[1] for id3 in valid_ideals[2]]   
    return [id1+id2+id3+id4 for id1 in valid_ideals[0] for id2 in valid_ideals[1] for id3 in valid_ideals[2] for id4 in valid_ideals[3]]           

In [ ]:
def get_valid_ideals(ideale, cubic, sing_cubics_factored):
    valid_ideals = []
    vrs = [c,e,f]
    for vr in vrs:
        valid_ideals_var = []
        del_ideale = ideale.elimination_ideal([vr])
        prim_deco_del = del_ideale.radical().primary_decomposition()
        for ideal in prim_deco_del:
            if is_ideal_valid(ideal, cubic, sing_cubics_factored):
                valid_ideals_var.append(ideal)
        if valid_ideals_var == []:
            return []
        else:
            valid_ideals.append(valid_ideals_var)
    return [id1+id2+id3 for id1 in valid_ideals[0] for id2 in valid_ideals[1] for id3 in valid_ideals[2]]   

In [20]:
with open('conditions.pickle', 'rb') as ff:
    conditions = pickle.load(ff)

In [21]:
len(conditions)

62

In [92]:
cond = conditions[1]
cond

[16*c^2 + d^2 + 16*c*e + 2*d*e + 5*e^2 - 2*d*f - 2*e*f + f^2, 32*c*d*e + d^2*e - 64*c*e^2 + 2*d*e^2 + 5*e^3 - 48*c*d*f + 6*d^2*f + 160*c*e*f - 6*d*e*f - 36*e^2*f - 48*c*f^2 - 12*d*f^2 + 101*e*f^2 + 6*f^3, 5*d^3 - 2*d^2*e - 32*c*e^2 + d*e^2 - 60*e^3 + 32*c*d*f - 13*d^2*f - 16*c*e*f + 40*d*e*f + 153*e^2*f + 64*c*f^2 + 11*d*f^2 - 134*e*f^2 - 3*f^3, 80*c*d^2 + 11*d^2*e - 304*c*e^2 + 22*d*e^2 + 55*e^3 - 176*c*d*f + 34*d^2*f + 1088*c*e*f - 130*d*e*f - 44*e^2*f - 192*c*f^2 - 68*d*f^2 + 407*e*f^2 + 34*f^3]

In [93]:
i=0
cond[i].variables(), [cond[i].degree(kk) for kk in cond[i].variables()]

((c, d, e, f), [2, 2, 2, 2])

In [94]:
for i in range(len(cond)):
    print(cond[i].variables(), [cond[i].degree(kk) for kk in cond[i].variables()])

(c, d, e, f) [2, 2, 2, 2]
(c, d, e, f) [1, 2, 3, 3]
(c, d, e, f) [1, 3, 3, 3]
(c, d, e, f) [1, 2, 3, 3]


In [42]:
cond[0].coefficient(d).factor()

(5) * f

In [47]:
cond[0].subs({d:0}).factor()

51*e^2 - 46*c*f - 39*e*f + 16*f^2

In [101]:
P.ideal(cond).elimination_ideal([e])

Ideal (25*c^4 + 10*c^3*d + 5*c^2*d^2 + 32*c^3*f + 10*c^2*d*f + 2*c*d^2*f + 22*c^2*f^2 - 2*c*d*f^2 + d^2*f^2 - 2*d*f^3 + f^4) of Multivariate Polynomial Ring in x, y, z, t, b, c, d, e, f, l, m over Rational Field

In [67]:
var('nv')
K1.<beta> = NumberField(9*nv^4-2*nv^2+1)

In [68]:
Q = PolynomialRing(K1, P.gens())

In [72]:
ideals = Q.ideal(cond).radical().primary_decomposition()
ideals

[Ideal (34*e + (-135*beta^3 + 9*beta^2 + 9*beta - 21)*f, 34*d + (-45*beta^3 - 99*beta^2 + 139*beta - 7)*f, 34*c + (-63*beta^3 + 45*beta^2 + 11*beta - 3)*f) of Multivariate Polynomial Ring in x, y, z, t, b, c, d, e, f, l, m over Number Field in beta with defining polynomial 9*nv^4 - 2*nv^2 + 1,
 Ideal (34*e + (-63*beta^3 - 9*beta^2 - 31*beta - 19)*f, 34*d + (387*beta^3 + 99*beta^2 - 101*beta - 29)*f, 34*c + (-9*beta^3 - 45*beta^2 - 19*beta + 7)*f) of Multivariate Polynomial Ring in x, y, z, t, b, c, d, e, f, l, m over Number Field in beta with defining polynomial 9*nv^4 - 2*nv^2 + 1,
 Ideal (34*e + (63*beta^3 - 9*beta^2 + 31*beta - 19)*f, 34*d + (-387*beta^3 + 99*beta^2 + 101*beta - 29)*f, 34*c + (9*beta^3 - 45*beta^2 + 19*beta + 7)*f) of Multivariate Polynomial Ring in x, y, z, t, b, c, d, e, f, l, m over Number Field in beta with defining polynomial 9*nv^4 - 2*nv^2 + 1,
 Ideal (34*e + (135*beta^3 + 9*beta^2 - 9*beta - 21)*f, 34*d + (45*beta^3 - 99*beta^2 - 139*beta - 7)*f, 34*c + (63*

In [80]:
ideals[0].gens()

[34*e + (-135*beta^3 + 9*beta^2 + 9*beta - 21)*f, 34*d + (-45*beta^3 - 99*beta^2 + 139*beta - 7)*f, 34*c + (-63*beta^3 + 45*beta^2 + 11*beta - 3)*f]

In [85]:
sost = solve_linear_system(ideals[0].gens(), [Q(c),Q(d),Q(e)], [Q(f)])
vrs = [Q(c),Q(d),Q(e),Q(f)]

In [91]:
Q(SE1.eqn).subs({vrs[i]:sost[i] for i in range(4)}).factor()

((12267564480*beta^3 - 817837632*beta^2 - 817837632*beta + 1908287808)) * f^2 * (x^2*y - x*y^2 + (-87/34*beta^3 + 33/34*beta^2 + 31/102*beta - 61/102)*x^2*z + (93/34*beta^3 - 15/17*beta^2 - 107/102*beta + 37/51)*x*y*z + (-15/34*beta^3 + 9/17*beta^2 + 37/102*beta - 4/17)*x*z^2 + (39/17*beta^3 - 6/17*beta^2 - 35/51*beta - 26/51)*x*y*t + (-36/17*beta^3 + 15/34*beta^2 - 1/17*beta + 65/102)*y^2*t + (-3/17*beta^3 - 3/34*beta^2 + 38/51*beta - 13/102)*y*z*t + (9/34*beta^3 - 21/34*beta^2 + 13/34*beta + 11/102)*y*t^2)

In [43]:
sing_cubics_SE1.factor()

(2) * (-c + f) * (-c + e) * (-e + f)^2 * (c + f)^2 * (c + e)^2 * c^5 * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f)^2 * (-c^3 + c^2*d - 3*c^2*f - c*d*f - c*e*f + e*f^2) * (-c^3 + c^2*d - 3*c^2*e - c*d*e - c*e*f + e^2*f) * (c^4 + 3*c^3*d - c^3*e + c^2*d*e - c^3*f + c^2*d*f - 6*c^2*e*f - c*d*e*f - c*e^2*f - c*e*f^2 + e^2*f^2)

In [45]:
lcm([pl.conditions for pl in SE1.tritangent_planes if pl.conditions !=0]).factor()

(-3*c^2 - c*e - c*f + e*f) * (-3*c^2 - c*d + e*f) * (-c^2 - 2*c*d + c*e + c*f + 3*e*f) * (c^2 - c*e + 3*c*f + e*f) * (c^2 + 3*c*e - c*f + e*f) * (c^2 - c*d + e*f) * (c^2*d - 2*c^2*e - c*d*e + c^2*f + e^2*f) * (c^2*d + c^2*e - 2*c^2*f - c*d*f + e*f^2) * (-c^3 + c*d*e - 2*c^2*f - c*d*f - c*e*f - e^2*f + e*f^2) * (-c^3 - 2*c^2*d - c*d*e - c*d*f + 3*c*e*f + e^2*f + e*f^2) * (-c^3 + c^2*d - 2*c^2*e - c^2*f - c*d*f - c*e*f + e*f^2) * (-c^3 + c^2*d - c^2*e - c*d*e - 2*c^2*f - c*e*f + e^2*f) * (c^3 + 2*c^2*e + c*d*e - c*d*f + c*e*f - e^2*f + e*f^2) * (c^3 + 3*c^2*d - 3*c^2*f - c*d*f - 3*c*e*f + e*f^2) * (c^3 + 3*c^2*d - 3*c^2*e - c*d*e - 3*c*e*f + e^2*f) * (-2*c^3*d + c^3*e - c^2*d*e + c^3*f - c^2*d*f + 5*c^2*e*f + c*d*e*f + c*e^2*f - c^2*f^2 - c*d*f^2 + c*e*f^2 - e^2*f^2 + e*f^3) * (c^3*d - c^3*e - c^3*f - 3*c^2*e*f - c*d*e*f + e^2*f^2) * (2*c^3*d - c^3*e + c^2*d*e + c^2*e^2 + c*d*e^2 - c^3*f + c^2*d*f - 5*c^2*e*f - c*d*e*f - c*e^2*f - e^3*f - c*e*f^2 + e^2*f^2) * (-3*c^4 - c^3*d - c^3*e + c^

In [22]:
P.ideal(conditions[0]).elimination_ideal([c]).groebner_basis()

[e^3 - 512/391*e^2*f + 63/391*d*f^2 + 332/391*e*f^2 - 33/391*f^3, d^2 + 243/23*e^2 - 10/23*d*f - 152/23*e*f + 37/23*f^2, d*e - 70/23*e^2 - 24/23*d*f + 63/23*e*f - 63/23*f^2]

In [24]:
nc = Cubic(SE1.eqn.subs({d:e-3*f, c:2*f-e}), line, sing_cubics_SE1.subs({d:e-3*f, c:2*f-e}))

In [25]:
nc.find_simmetries(nc.find_admissible_projectivities())

[
[1 0 0 0]  [-e^2 + 4*e*f - 4*f^2                    0                    0   -e^2 + 2*e*f - f^2]
[0 1 0 0]  [                   0 -e^2 + 4*e*f - 4*f^2                    0        2*e*f - 4*f^2]
[0 0 1 0]  [                   0                    0 -e^2 + 4*e*f - 4*f^2  e^2 - 4*e*f + 5*f^2]
[0 0 0 1], [                   0                    0                    0  e^2 - 4*e*f + 4*f^2]
]